In [ ]:
with open("Jakub_Bliźniuk_Bartłomiej_Dmitruk.txt", "w", encoding="utf-8") as f:
    f.write("To wiadomość do zaszyfrowania")

# Biblioteki kryptograficzne w Pythonie
## PyCryptodome

## Zadanie 1
### Działania na ciałach skończonych

W pythonie bardzo łatwo jest impelemntować działania w ciałach skończonych po prostu używając operatora modulo (`%`) - np.

In [ ]:
(10+8)%13

In [ ]:
(10-14)%13

In [ ]:
(8*3)%13

  Standardowa funkcja do potęgowania pozwala także znaleźć element odwrotny:

In [ ]:
pow(8, -1, 13)

Warto zauważyć, że jeśli mamy inną liczbę elementów niż jakaś potęga liczby pierwszej, to istnieją (niezerowe) elementy nieodwracalne

In [ ]:
pow(10, -1, 15)

Zwykle więc będą nas interesować tylko skończone ciała pierwsze. Do sprawdzenia pierwszości możemy np. wykorzystać metodę `isPrime` z PyCryptoDome

In [ ]:
from Cryptodome.Util.number import isPrime
isPrime(13)

Dostaniemy też odpowiedni wynik gdy sprawdzamy liczbę która nie jest pierwsza:

In [ ]:
isPrime(15)

Tego rodzaju arytmetyka jest podstawą wielu kryptosystemów (np. RSA).
Jednak zwykle potrzebujemy większych liczb pierwszych. Możemy je wygenerować np. używając getPrime:

In [ ]:
from Cryptodome.Util.number import getPrime
getPrime(1024)

I jak możemy zobaczyć, nasze operacje bez problemu sobie z tym radzą:

In [ ]:
prime = getPrime(1024)
(1000+10**1000)%prime

In [ ]:
(1000-10**1000)%prime

In [ ]:
(1000*10**1000)%prime

In [ ]:
pow(100, -1, prime)

A jak ta funkcja generuje te liczby pierwsze? Definicja jest dość prosta

In [ ]:
from Cryptodome import Random
from Cryptodome.Util.number import getRandomNBitInteger
def getPrime(N, randfunc=None):
	"""Return a random N-bit prime number.

	N must be an integer larger than 1.
	If randfunc is omitted, then :meth:`Random.get_random_bytes` is used.
	"""
	if randfunc is None:
		randfunc = Random.get_random_bytes
	
	if N < 2:
		raise ValueError("N must be larger than 1")

	while True:
		number = getRandomNBitInteger(N, randfunc) | 1
		if isPrime(number, randfunc=randfunc):
			break
	return number

Ale jak ta funkcja generuje liczbę losową?

## Zadanie 2
### Generowanie liczb losowych

Python ma w zasadzie trzy wbudowane biblioteki do generowania liczb losowych - `random`, `secrets` is `os.urandom`

`random` to generator pseudolosowy (MT19937), a `secrets` is `os.urandom` korzystają z systemowej losowości

`random` oferuje kilka użytecznych funkcji. Na przykład:

In [ ]:
from random import choice, choices, getrandbits, random, randrange, gauss
choice("abcdefg")

In [ ]:
choices([1,2,3,4,5], k=3)

In [ ]:
getrandbits(32)

In [ ]:
random()

In [ ]:
randrange(10, 100)

In [ ]:
gauss()

`secrets` oferuje mniej funkcji - w zasadzie tylko 3 podobne do tych z random:

In [ ]:
from secrets import choice as secrets_choice, randbelow, randbits
secrets_choice("abcdefg")

In [ ]:
randbelow(100)

In [ ]:
randbits(32)

Dodatkowo mamy jednak funkcje do generowania tokenów - ciągów bajtów, opcjonalnie z właściwym kodowaniem

In [ ]:
from secrets import token_bytes, token_hex, token_urlsafe
token_bytes(32)

In [ ]:
token_hex(32)

In [ ]:
token_urlsafe(32)

`os` oferuje w zasadzie tylko jedną funkcję - `urandom` (na Linuxie jest jeszcze dość nowe `os.getrandom`, ale nie jest to dostępne na innych platformach). Także korzysta ona z systemowej losowości. Działa tak samo jak `token_bytes` (które z niej właśnie korzysta):

In [ ]:
from os import urandom
urandom(32)

Wracając do PyCryptodome, moduł `Cryptodome.Random` zawiera `get_random_bytes`, które jest obecnie po prostu aliasem dla `os.urandom` i robi dokładnie to samo:

In [ ]:
from Cryptodome.Random import get_random_bytes
get_random_bytes(32)

### A co z naszym `getPrime`?

Korzysta z `Cryptodome.Random.get_random_bytes`, czyli w praktyce z `os.urandom`

## Do wszystkiego wymagającego bezpiecznych liczb losowych należy **UNIKAĆ** random

Dlaczego? Bo da się łatwo przewidzieć wyniki! Wystarczy 624 kolejne liczby by **w pełni** sklonować generator

Możemy do tego wykorzystać moduł RandCrack:

In [ ]:
from randcrack import RandCrack
rc = RandCrack()
for i in range(624):
	rc.submit(getrandbits(32))

In [ ]:
print(f"przewidziane: {rc.predict_getrandbits(32)}, losowe: {getrandbits(32)}")

Możemy też przewidzieć inne funkcje z `random`!

In [ ]:
print(f"przewidziana: {rc.predict_random()}, losowa: {random()}")
print(f"przewidziana: {rc.predict_randrange(10, 100)}, losowa: {randrange(10, 100)}")
print(f"przewidziana: {rc.predict_choice('abcdefg')}, losowa: {choice('abcdefg')}")

Oczywiście nie działa to z `secrets`:

In [ ]:
print(f"przewidziane: {rc.predict_choice('abcdefg')}, losowe: {secrets_choice('abcdefg')}")

nawet gdy zasilimy generator z `secrets`

In [ ]:
rcs=RandCrack()
for i in range(624):
	rcs.submit(randbits(32))
print(f"przewidziane: {rcs.predict_getrandbits(32)}, losowe: {randbits(32)}")

### Użycie PyCryptodome 

W tym pakiecie znajdują się dwie funkcje, które generują losowe liczby pierwsze:

- getPrime()  : returns a random N-bit prime number

In [ ]:
from Cryptodome.Util.number import getPrime

In [ ]:
random_prime = getPrime(128)
print(f"{'  Random prime (128 bits): ':<35} {random_prime}")

In [ ]:
from Cryptodome.Util.number import getStrongPrime

- getStrongPrime() : returns a random strong N-bit prime number.

W tym kontekście p jest silną liczbą pierwszą, jeśli p-1 i p+1 mają co najmniej jeden duży czynnik pierwszy

In [ ]:
strong_random_prime = getStrongPrime(512)
print(f"{'  Strong random prime (512 bits): ':<35} {strong_random_prime}")

## Zadanie 3
### Szyfrowanie AES

PyCryptodome obsługuje szyfr AES w klasie `AES` w `Cryptodome.Cipher`

In [ ]:
from Cryptodome.Cipher import AES

W celu zaszyfrowania czegokolwiek musimy jednak najpierw wygenerować klucz - możemy do tego użyć metod z poprzedniego zadania, np. z `secrets`:

In [ ]:
from secrets import token_bytes
key = token_bytes(16)
key

Musimy też wczytać dane do zaszyfrowania, do czego możemy użyć po prostu metod wbudowanych w Pythona.

In [ ]:
with open("Jakub_Bliźniuk_Bartłomiej_Dmitruk.txt", "rb") as f:
	data = f.read()
data

Ponieważ AES pracuje na blokach, musimy się upewnić, że nasze dane mają długość wielokrotności rozmiaru bloku. PyCryptodome dostarcza do tego metody `pad`:

In [ ]:
from Cryptodome.Util.Padding import pad
padded_data = pad(data, AES.block_size)
padded_data

Zostało już tylko stworzyć obiekt klasy i możemy szyfrować:

In [ ]:
cipher = AES.new(key, AES.MODE_CBC)
ciphertext = cipher.encrypt(padded_data)
ciphertext

warto zauważyć, że używamy tutaj demonstracyjnie trybu CBC (zamiast bardziej nowoczesnych), do którego automatycznie generujemy IV - które dość prosto odzyskać z obiektu szyfru:

In [ ]:
cipher.iv

Odszyfrowanie jest równie proste:

In [ ]:
decryption_cipher = AES.new(key, AES.MODE_CBC, iv=cipher.iv)
decrypted_data = decryption_cipher.decrypt(ciphertext)
decrypted_data

Choć też musimy skorzystać z funkcji do paddingu by odzyskać rzeczywisty oryginał:

In [ ]:
from Cryptodome.Util.Padding import unpad
unpad(decrypted_data, AES.block_size)

Przy czym jest to obiekt `bytes`, z co widzimy po dziwnym przedstawieniu znaków z Unicode. By odzyskać prostego stringa musimy po prostu odkodować string:

In [ ]:
unpad(decrypted_data, AES.block_size).decode()

### Wprowadzanie błędów

### Co się dzieje gdy klucz jest niepoprawny?

In [ ]:
from Cryptodome.Util.strxor import strxor
invalid_key_cipher = AES.new(strxor(key, b'just some errors') , AES.MODE_CBC, iv=cipher.iv)
invalid_key_data = invalid_key_cipher.decrypt(ciphertext)
invalid_key_data


Jak widać, otrzymujemy zupełnie złe dane... Nawet usuwanie paddingu nie działa przez to, że jest inny niż w oryginale:

In [ ]:
unpad(invalid_key_data, AES.block_size)

### A co ze złym tekstem?

Dodając błąd na początku widzimy tu nawet kawałek tekstu źródłowego:

In [ ]:
invalid_ciphertext = b'\x00' + ciphertext[1:]
decryption_cipher.decrypt(invalid_ciphertext)

Jeśli jednak zrobimy to na końcu lub środku przynajmniej na pierwszy rzut oka dostajemy niezwiązany tekst:

In [ ]:
invalid_ciphertext = ciphertext[:-1] + b'\x00'
decryption_cipher.decrypt(invalid_ciphertext)

In [ ]:
invalid_ciphertext = ciphertext[:-10] + b'\x00' + ciphertext[-9:]
decryption_cipher.decrypt(invalid_ciphertext)


To jak będą wyglądały błędy będzie jednak w pełni zależeć od wybranego trybu szyfrowania - gdybyśmy korzystali z ECB na przykład, błąd dotykał by tylko bloku w którym się znajduje, ponieważ bloki nie są od siebie zależne

## Zadanie 4
### Funkcja skrótu

Wykorzystując plik z poprzedniego ćwiczenia obliczamy z niego skrót.

PyCryptodome umożliwia obliczanie skrótów za pomocą wielu funkcji skrótu. W celach prezentacji użyliśmy trzech funkcji:
- SHA
- SHA3
- MD5

In [ ]:
from Cryptodome.Hash import SHA256
from Cryptodome.Hash import SHA3_256
from Cryptodome.Hash import MD5

hash_MD5 = MD5.new()
hash_SHA_256 = SHA256.new()
hash_SHA3_256 = SHA3_256.new()

Do ćwiczenia wykorzystujemy plik z poprzedniego zadania: 

In [ ]:
filename = "Jakub_Bliźniuk_Bartłomiej_Dmitruk.txt"

Aby umożliwiać wielokrotne wykorzystywanie kodu nadpisujemy tekst:

In [ ]:
with open(filename, "w", encoding="utf-8") as f:
    f.write("To wiadomość do zaszyfrowania")

Następnym krokiem jest odczytywanie pliku blokami i stworzenie skrótu:

In [ ]:
with open(filename, "rb") as f:
	while data := f.read(1024):
		hash_MD5.update(data)
		hash_SHA_256.update(data)
		hash_SHA3_256.update(data)

By poznać skrót musimy użyć metody `digest` lub `hexdigest` - gdzie ta pierwsza zwraca nam dane binarne, a druga hexydecymalny string:

In [ ]:
hash_MD5.digest()

In [ ]:
print("MD5:      ", hash_MD5.hexdigest())
print("SHA-256:  ", hash_SHA_256.hexdigest())
print("SHA3-256: ", hash_SHA3_256.hexdigest())

Następnie zmieniamy treść pliku na: "Ta wiadomość do zaszyfrowania"

In [ ]:
with open(filename, "w", encoding="utf-8") as f:
    f.write("Ta wiadomość do zaszyfrowania")

Analogicznie obliczamy skróty dla nowej wersji pliku.

In [ ]:
hash2_SHA_256 = SHA256.new()
hash2_MD5 = MD5.new()
hash2_SHA3_256 = SHA3_256.new()

with open(filename, "rb") as f:
	while data := f.read(1024):
		hash2_SHA_256.update(data)
		hash2_MD5.update(data)
		hash2_SHA3_256.update(data)
print("MD5:      ", hash2_MD5.hexdigest())
print("SHA-256:  ", hash2_SHA_256.hexdigest())
print("SHA3-256: ", hash2_SHA3_256.hexdigest())

Następnie porównujemy wartości hashy w oryginalnym i zmodyfikowanym pliku.

In [ ]:
if hash_MD5.hexdigest() == hash2_MD5.hexdigest():
    print(f"MD5 są równe: {hash_MD5.hexdigest()}")
else:
    print(f"MD5 są różne: {hash_MD5.hexdigest()} i {hash2_MD5.hexdigest()}")
if hash_SHA_256.hexdigest() == hash2_SHA_256.hexdigest():
    print(f"SHA-256 są równe: {hash_SHA_256.hexdigest()}")
else:
    print(f"SHA-256 są różne: {hash_SHA_256.hexdigest()} i {hash2_SHA_256.hexdigest()}")

if hash_SHA3_256.hexdigest() == hash2_SHA3_256.hexdigest():
    print(f"SHA3-256 są równe: {hash_SHA3_256.hexdigest()}")
else:
    print(f"SHA3-256 są rózne: {hash_SHA3_256.hexdigest()} i {hash2_SHA3_256.hexdigest()}")

# Koniec

### Autorzy: 
#### Jakub Bliźniuk i Bartłomiej Dmitruk

Czyszczenie po zmianach w plikach:

In [ ]:
with open(filename, "w", encoding="utf-8") as f:
    f.write("To wiadomość do zaszyfrowania")